## Calculate temperature profiles under different modelling assumptions

In [1]:
# Set directory
ROOT_DIR = abspath(joinpath(pwd(),"../../"))
if !haskey(ENV, "RAD_DIR")
    ENV["RAD_DIR"] = joinpath(ROOT_DIR,"socrates")
end

In [2]:
# Import system packages
using Pkg
using Revise

using Printf
using DelimitedFiles
using Plots
default(label=nothing, grid=true)

In [3]:
using LaTeXStrings

In [4]:
# Import AGNI
Pkg.activate(ROOT_DIR)
using AGNI

# Normal logging from AGNI module
AGNI.setup_logging("",2)

  Activating project at `~/AGNI`


In [5]:
R_earth = 6.371e6 # m
S_earth = 1361.0 # W/m^2

1361.0

### Define atmosphere parameters

In [6]:
# Configuration options
instellation    = 2 * S_earth  # Solar flux [W m-2]
gravity         = 9.81
radius          = 1 * R_earth
zenith          = 54.74
albedo          = 0.0
s0_fact         = 0.25
nlev_centre     = 45
p_surf          = 300.0     # bar
t_surf          = 2000.0
p_top           = 1e-5      # bar
mole_fractions  = Dict([ ("H2O", 0.4), ("H2", 0.6)])

spectral_file = joinpath(ROOT_DIR,"res/spectral_files/Dayspring/48/Dayspring.sf")
star_file     = joinpath(ROOT_DIR,"res/stellar_spectra/sun.txt");


In [7]:
# Create output directory
output_dir = joinpath(ROOT_DIR,"out")
rm(output_dir,force=true,recursive=true)
mkdir(output_dir);

### Initialise the atmosphere

In [ ]:
# Setup atmosphere
atmos = atmosphere.Atmos_t()
atmosphere.setup!(atmos, ROOT_DIR, output_dir,
                        spectral_file,
                        instellation, s0_fact, albedo, zenith,
                        t_surf,
                        gravity, radius,
                        nlev_centre, p_surf, p_top,
                        mole_fractions, "",
                        flag_gcontinuum=true,
                        flag_rayleigh=true,
                        thermo_functions=true
                        )
atmosphere.allocate!(atmos, star_file)
setpt.isothermal!(atmos, t_surf)

In [ ]:
atmos_results::Dict{String, atmosphere.Atmos_t} = Dict{String, atmosphere.Atmos_t}()

## Run the model under various scenarios

### Fixed surface temperature

In [ ]:
atmos.tmp_surf = deepcopy(t_surf)
println(atmos.tmp_surf)

In [ ]:
# Isothermal
setpt.isothermal!(atmos, atmos.tmp_surf)
atmosphere.calc_layer_props!(atmos)
energy.calc_fluxes!(atmos, true, false, true, false, false)
atmosphere.calc_observed_rho!(atmos)
atmos_results["1_iso"] = deepcopy(atmos)
;

In [ ]:
# Adiabatic
setpt.dry_adiabat!(atmos)
atmosphere.calc_layer_props!(atmos)
energy.calc_fluxes!(atmos, true, false, true, false, false)
atmosphere.calc_observed_rho!(atmos)
atmos_results["1_ad"] = deepcopy(atmos)
;

In [ ]:
# Adiabatic + stratosphere
setpt.dry_adiabat!(atmos)
setpt.stratosphere!(atmos, phys.calc_Tskin(atmos.instellation, atmos.albedo_b))
atmosphere.calc_layer_props!(atmos)
energy.calc_fluxes!(atmos, true, false, true, false, false)
atmosphere.calc_observed_rho!(atmos)
atmos_results["1_ad+strat"] = deepcopy(atmos)
;

In [ ]:
# Radiative-convective
setpt.isothermal!(atmos, t_surf) # initial guess
succ = solver.solve_energy!(atmos,
                                    sol_type=1,
                                    method=1,
                                    dx_max=200.0,
                                    ls_method=2,
                                    modplot=0,
                                    save_frames=false,
                                    detect_plateau=false
                                    )
println("Solver success? $succ")
atmosphere.calc_observed_rho!(atmos)
atmos_results["1_radconv"] = deepcopy(atmos)
;

### Solving for energy balance

In [ ]:
# Isothermal
succ = solver.solve_prescribed!(atmos, sol_type=3, atm_type=1, tmp_upper=1000.0)
println("Solver success? $succ")
atmosphere.calc_observed_rho!(atmos)
atmos_results["3_iso"] = deepcopy(atmos)
;

In [ ]:
# Adiabatic
succ = solver.solve_prescribed!(atmos, sol_type=3, atm_type=2)
println("Solver success? $succ")
atmosphere.calc_observed_rho!(atmos)
atmos_results["3_ad"] = deepcopy(atmos)
;

In [ ]:
# Adiabatic + stratosphere
succ = solver.solve_prescribed!(atmos, sol_type=3, atm_type=3)
println("Solver success? $succ")
atmosphere.calc_observed_rho!(atmos)
atmos_results["3_ad+strat"] = deepcopy(atmos)
;

In [ ]:
# Radiative-convective
setpt.isothermal!(atmos, t_surf) # initial guess
succ = solver.solve_energy!(atmos,
                                    sol_type=3,
                                    method=1,
                                    dx_max=200.0,
                                    ls_method=2,
                                    modplot=0,
                                    save_frames=false,
                                    detect_plateau=false
                                    )
println("Solver success? $succ")
atmosphere.calc_observed_rho!(atmos)
atmos_results["3_radconv"] = deepcopy(atmos)
;

In [ ]:
# save to netcdf
for (k,atm) in atmos_results
    save.write_ncdf(atm, joinpath(output_dir,k*".nc"))
end


In [ ]:
fig = plot(legend=:right, size=(700,600),
            ylabel="Radius [Earth radii]",
            xlabel="Temperature [K]", legendfontsize=10, labelfontsize=12, tickfontsize=12, frame=:box)

lw = 2.5

Teqm = phys.calc_Tskin(atmos.instellation, atmos.albedo_b)
vline!(fig, [Teqm], ls=:dash, color=:silver, lw=lw, label=@sprintf("Skin temperature"))

# default axis limits
y_min = 10.0
y_max = 0.5
x_min = -500
x_max = 10.0


lstyles = Dict([("iso",:dot), ("ad",:dash), ("ad+strat",:dashdot), ("radconv",:solid)])

# plot
for (k,atm) in atmos_results

    spl = split(k,"_")

    # line label and color
    if spl[1] == "1"
        col = "orange"
    else
        col = "blue"
    end
    ls = lstyles[spl[2]]

    # plot profile
    plot!(fig, atm.tmp, atm.r / R_earth, lw=lw, linestyle=ls, color=col)

    # plot photosphere point
    atmosphere.calc_observed_rho!(atm)
    rtrans = atm.transspec_r/R_earth
    plot!(fig, [x_min, atm.transspec_tmp], [rtrans,rtrans], color=col, linealpha=0.5, linestyle=ls)
    scatter!(fig, [atm.transspec_tmp], [rtrans], color=col)
    scatter!(fig, [x_min], [rtrans], color=col)

    # axis limits
    y_min = min( minimum(atm.r) / R_earth, y_min)
    y_max = max( rtrans, y_max)
    x_max = max( maximum(atm.tmp), x_max)
end

# legend entries
llw = 2
plot!(fig, [], [], label=" ", linealpha=0) # spacer
plot!(fig, [], [], lw=llw, label=L"$T$ boundary condition", lc="orange", ls=:solid,  )
plot!(fig, [], [], lw=llw, label=L"$F$ boundary condition", lc="blue",   ls=:solid,  )
plot!(fig, [], [], label=" ", linealpha=0) # spacer
plot!(fig, [], [], lw=llw, label="Isothermal",           lc="black",  ls=:dot,    )
plot!(fig, [], [], lw=llw, label="Adiabatic",            lc="black",  ls=:dash,   )
plot!(fig, [], [], lw=llw, label="Adiabatic + skin",     lc="black",  ls=:dashdot,)
plot!(fig, [], [], lw=llw, label="Radiative-convective", lc="black",  ls=:solid,)

y_max += 0.05
yaxis!(ylim=(1.0, y_max), yticks=range(1.0, y_max, step=0.05))
xaxis!(xlim=(x_min, x_max), xticks=range(0.0, x_max, step=500))

display(fig)
savefig(fig, joinpath(output_dir,"plot_height.pdf"))